# Extração dos arquivos do DFP e ITR

* https://dados.cvm.gov.br/dataset/cia_aberta-doc-dfp
* https://dados.cvm.gov.br/dataset/cia_aberta-doc-itr

In [1]:
from pathlib import Path
import glob
import os
import pandas as pd
import re
import zipfile

## DFP

In [ ]:
# Ano escolhido
ano = '2024'

# Nome do arquivo zipado
zip_filename = f'C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta_zip//dfp_cia_aberta_{ano}.zip'

# Lista dos nomes dos arquivos que você deseja extrair
arquivos_a_extrair = [
    # Consolidado
    f'dfp_cia_aberta_BPA_con_{ano}.csv',
    f'dfp_cia_aberta_BPP_con_{ano}.csv',
    f'dfp_cia_aberta_DFC_MD_con_{ano}.csv',
    f'dfp_cia_aberta_DFC_MI_con_{ano}.csv',
    f'dfp_cia_aberta_DMPL_con_{ano}.csv',
    f'dfp_cia_aberta_DRA_con_{ano}.csv',
    f'dfp_cia_aberta_DRE_con_{ano}.csv',
    f'dfp_cia_aberta_DVA_con_{ano}.csv',
    # Individual
    f'dfp_cia_aberta_BPA_ind_{ano}.csv',
    f'dfp_cia_aberta_BPP_ind_{ano}.csv',
    f'dfp_cia_aberta_DFC_MD_ind_{ano}.csv',
    f'dfp_cia_aberta_DFC_MI_ind_{ano}.csv',
    f'dfp_cia_aberta_DMPL_ind_{ano}.csv',
    f'dfp_cia_aberta_DRA_ind_{ano}.csv',
    f'dfp_cia_aberta_DRE_ind_{ano}.csv',
    f'dfp_cia_aberta_DVA_ind_{ano}.csv',
]

# Caminho do diretório do arquivo não zipado
destino_dir = f'C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//{ano}'

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    for arquivo in arquivos_a_extrair:
        zip_ref.extract(arquivo, destino_dir)

        csv_path = f'{destino_dir}//{arquivo}'
        parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

        # Tentativa de leitura com fallback
        try:
            df = pd.read_csv(
                csv_path, sep=';', encoding='ISO-8859-1',
                low_memory=False, engine='c'
            )
        except Exception as e:
            print(f"[AVISO] Problema ao ler {arquivo} com engine='c': {e}")
            print("→ Tentando novamente com engine='python' e on_bad_lines='warn'")
            df = pd.read_csv(
                csv_path, sep=';', encoding='ISO-8859-1',
                engine='python', on_bad_lines='warn'
            )

        # Salvando em parquet
        df.to_parquet(parquet_path)

# Entrando no diretório especificado
os.chdir(destino_dir)

# Encontrando todos os arquivos .csv no diretório
arquivos_csv = glob.glob('*.csv')

for arquivo in arquivos_csv:
    os.remove(arquivo)

In [10]:
# Compilando cada arquivo parquet em apenas um arquivo

# Configurações 
base_dir = Path(r'C:/B3/historico-arquivos/dfp-b3/dfp-cia-aberta')
years = list(range(2017, 2025))  # Lista dos anos de 2017 até 2024
output_dir = base_dir / 'consolidados'
output_dir.mkdir(parents=True, exist_ok=True)

# Encontrarndo todos os arquivos parquet nas pastas de cada ano
parquet_paths = []
for y in years:
    year_dir = base_dir / str(y)
    if not year_dir.exists():
        print(f'[AVISO] Diretório {year_dir} não encontrado — pulando.')
        continue
    parquet_paths.extend(list(year_dir.glob("*.parquet")))

if not parquet_paths:
    raise SystemExit('Nenhum arquivo parquet encontrado nas pastas de ano especificadas.')

# Agrupando por 'base name' (removendo o sufixo do ano se existir)
groups = {}
for p in parquet_paths:
    stem = p.stem  # ex: 'dfp_cia_aberta_BPA_con_2017'
    parts = stem.rsplit("_", 1)  # separa no último underscore
    if len(parts) == 2 and re.fullmatch(r"\d{4}", parts[1]):  # sufixo é um ano
        base_name = parts[0]
    else:
        # se não terminar em ano, usa o stem completo
        base_name = stem

    groups.setdefault(base_name, []).append(p)

# Para cada grupo, lendo todos os parquet e concatenando
for basename, paths in groups.items():
    # Ordenando por caminho (opcional: por ano)
    paths_sorted = sorted(paths, key=lambda x: x.name)
    print(f"\n[INFO] Agrupando {len(paths_sorted)} arquivos para '{basename}':")
    for pp in paths_sorted:
        print('  -', pp)

    dfs = []
    for pp in paths_sorted:
        try:
            df = pd.read_parquet(pp)
            dfs.append(df)
        except Exception as e:
            print(f'[ERRO] Falha ao ler {pp}: {e}. Pulando este arquivo.')
            continue

    if not dfs:
        print(f'[AVISO] Nenhum dataframe válido para {basename}, pulando.')
        continue

    # Concat — se colunas diferirem, concat preencherá com NaN
    try:
        df_all = pd.concat(dfs, ignore_index=True, sort=False)
    except Exception as e:
        print(f'[ERRO] Erro ao concatenar dataframes para {basename}: {e}')
        # tentativa alternativa: alinhar colunas manualmente
        common_cols = sorted({c for df in dfs for c in df.columns})
        dfs_aligned = [df.reindex(columns=common_cols) for df in dfs]
        df_all = pd.concat(dfs_aligned, ignore_index=True, sort=False)

    # Nome final do arquivo: 'basename_2017_2024.parquet'
    out_name = f'{basename}_{years[0]}_{years[-1]}.parquet'
    out_path = output_dir / out_name

    try:
        df_all.to_parquet(out_path, index=False)
        print(f'[OK] Gravado {out_path} (linhas: {len(df_all)}, colunas: {len(df_all.columns)})')
    except Exception as e:
        print(f'[ERRO] Falha ao salvar {out_path}: {e}')

print('\n[FINALIZADO] Todos os agrupamentos processados.')



[INFO] Agrupando 8 arquivos para 'dfp_cia_aberta_BPA_con':
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2017\dfp_cia_aberta_BPA_con_2017.parquet
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2018\dfp_cia_aberta_BPA_con_2018.parquet
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2019\dfp_cia_aberta_BPA_con_2019.parquet
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2020\dfp_cia_aberta_BPA_con_2020.parquet
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2021\dfp_cia_aberta_BPA_con_2021.parquet
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2022\dfp_cia_aberta_BPA_con_2022.parquet
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2023\dfp_cia_aberta_BPA_con_2023.parquet
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2024\dfp_cia_aberta_BPA_con_2024.parquet
[OK] Gravado C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\consolidados\dfp_cia_aberta_BPA_con_2017_2024.parquet (linhas: 444849, colunas: 14)

[INFO] Agrupando 8 arquivos para 'dfp_cia_aberta_BPA_ind':


In [ ]:
# O arquivo 'dfp_cia_aberta_composicao_capital_{ano}.csv' está disponível a partir de 2020

# Ano escolhido
ano = '2024'

# Nome do arquivo zipado
zip_filename = f'C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta_zip//dfp_cia_aberta_{ano}.zip'

# Lista dos nomes dos arquivos que você deseja extrair
arquivos_a_extrair = [
    # Composição de capital
    f'dfp_cia_aberta_composicao_capital_{ano}.csv'
]

# Caminho do diretório do arquivo não zipado
destino_dir = f'C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//{ano}'

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    for arquivo in arquivos_a_extrair:
        zip_ref.extract(arquivo, destino_dir)

        csv_path = f'{destino_dir}//{arquivo}'
        parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

        # Tentativa de leitura com fallback
        try:
            df = pd.read_csv(
                csv_path, sep=';', encoding='ISO-8859-1',
                low_memory=False, engine='c'
            )
        except Exception as e:
            print(f"[AVISO] Problema ao ler {arquivo} com engine='c': {e}")
            print("→ Tentando novamente com engine='python' e on_bad_lines='warn'")
            df = pd.read_csv(
                csv_path, sep=';', encoding='ISO-8859-1',
                engine='python', on_bad_lines='warn'
            )

        # Salvando em parquet
        df.to_parquet(parquet_path)

# Entrando no diretório especificado
os.chdir(destino_dir)

# Encontrando todos os arquivos .csv no diretório
arquivos_csv = glob.glob('*.csv')

for arquivo in arquivos_csv:
    os.remove(arquivo)

In [17]:
# Compilando o arquivo 'dfp_cia_aberta_composicao_capital_{ano}.csv' que está disponível a partir de 2020

# Configurações
base_dir = Path(r'C:/B3/historico-arquivos/dfp-b3/dfp-cia-aberta')
years = list(range(2020, 2025))  # Lista dos anos de 2020 até 2024
output_dir = base_dir / 'consolidados'
output_dir.mkdir(parents=True, exist_ok=True)

# Defina o padrão de nome de arquivo desejado (sem o ano)
# O nome do arquivo será: dfp_cia_aberta_composicao_capital_{ano}.parquet
FILE_BASE_NAME = 'dfp_cia_aberta_composicao_capital'

# Encontrarndo apenas os arquivos 'dfp_cia_aberta_composicao_capital_{ano}.parquet' nas pastas de cada ano
parquet_paths = []
for y in years:
    year_dir = base_dir / str(y)
    if not year_dir.exists():
        print(f'[AVISO] Diretório {year_dir} não encontrado — pulando.')
        continue

    # >>> MODIFICAÇÃO CHAVE AQUI <<<
    # Procuramos o arquivo específico pelo nome, que inclui o ano
    file_name = f'{FILE_BASE_NAME}_{y}.parquet'
    file_path = year_dir / file_name

    if file_path.exists():
        parquet_paths.append(file_path)
    else:
        print(f'[AVISO] Arquivo {file_name} não encontrado em {year_dir} — pulando.')


if not parquet_paths:
    raise SystemExit('Nenhum arquivo parquet encontrado nas pastas de ano especificadas.')

# Agrupando por 'base name' (removendo o sufixo do ano se existir)
# *O código de agrupamento não precisa de mudanças, mas será simplificado já que agora só temos 1 tipo de arquivo.*
groups = {}
for p in parquet_paths:
    # O stem agora é sempre 'dfp_cia_aberta_composicao_capital_{ano}'
    stem = p.stem
    # Remove o '_2017' (ou outro ano) para ter o nome base
    parts = stem.rsplit("_", 1)
    # Assumindo que o sufixo é sempre o ano e queremos o nome base 'dfp_cia_aberta_composicao_capital'
    if len(parts) == 2 and re.fullmatch(r"\d{4}", parts[1]):
        base_name = parts[0] # Ex: 'dfp_cia_aberta_composicao_capital'
    else:
        base_name = stem # Caso o nome não termine em ano (improvável aqui, mas mantém a lógica original)

    groups.setdefault(base_name, []).append(p)

# Para cada grupo, lendo todos os parquet e concatenando
for basename, paths in groups.items():
    # ... (O restante do código de processamento permanece inalterado)

    # Ordenando por caminho (opcional: por ano)
    paths_sorted = sorted(paths, key=lambda x: x.name)
    print(f"\n[INFO] Agrupando {len(paths_sorted)} arquivos para '{basename}':")
    for pp in paths_sorted:
        print('  -', pp)

    dfs = []
    for pp in paths_sorted:
        try:
            df = pd.read_parquet(pp)
            dfs.append(df)
        except Exception as e:
            print(f'[ERRO] Falha ao ler {pp}: {e}. Pulando este arquivo.')
            continue

    if not dfs:
        print(f'[AVISO] Nenhum dataframe válido para {basename}, pulando.')
        continue

    # Concat — se colunas diferirem, concat preencherá com NaN
    try:
        df_all = pd.concat(dfs, ignore_index=True, sort=False)
    except Exception as e:
        print(f'[ERRO] Erro ao concatenar dataframes para {basename}: {e}')
        # tentativa alternativa: alinhar colunas manualmente
        common_cols = sorted({c for df in dfs for c in df.columns})
        dfs_aligned = [df.reindex(columns=common_cols) for df in dfs]
        df_all = pd.concat(dfs_aligned, ignore_index=True, sort=False)

    # Nome final do arquivo: 'basename_2017_2024.parquet'
    out_name = f'{basename}_{years[0]}_{years[-1]}.parquet'
    out_path = output_dir / out_name

    try:
        df_all.to_parquet(out_path, index=False)
        print(f'[OK] Gravado {out_path} (linhas: {len(df_all)}, colunas: {len(df_all.columns)})')
    except Exception as e:
        print(f'[ERRO] Falha ao salvar {out_path}: {e}')

print('\n[FINALIZADO] Todos os agrupamentos processados.')


[INFO] Agrupando 5 arquivos para 'dfp_cia_aberta_composicao_capital':
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2020\dfp_cia_aberta_composicao_capital_2020.parquet
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2021\dfp_cia_aberta_composicao_capital_2021.parquet
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2022\dfp_cia_aberta_composicao_capital_2022.parquet
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2023\dfp_cia_aberta_composicao_capital_2023.parquet
  - C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\2024\dfp_cia_aberta_composicao_capital_2024.parquet
[OK] Gravado C:\B3\historico-arquivos\dfp-b3\dfp-cia-aberta\consolidados\dfp_cia_aberta_composicao_capital_2020_2024.parquet (linhas: 3666, colunas: 10)

[FINALIZADO] Todos os agrupamentos processados.


## ITR

In [23]:
# Ano escolhido
ano = '2025'    

# Nome do arquivo zipado
zip_filename = f'C://B3//historico-arquivos//itr-b3//itr-cia-aberta_zip//itr_cia_aberta_{ano}.zip'

# Lista dos nomes dos arquivos que você deseja extrair
arquivos_a_extrair = [
    # Consolidado
    f'itr_cia_aberta_BPA_con_{ano}.csv',
    f'itr_cia_aberta_BPP_con_{ano}.csv',
    f'itr_cia_aberta_DFC_MD_con_{ano}.csv',
    f'itr_cia_aberta_DFC_MI_con_{ano}.csv',
    f'itr_cia_aberta_DMPL_con_{ano}.csv',
    f'itr_cia_aberta_DRA_con_{ano}.csv',
    f'itr_cia_aberta_DRE_con_{ano}.csv',
    f'itr_cia_aberta_DVA_con_{ano}.csv',
    # Individual
    f'itr_cia_aberta_BPA_ind_{ano}.csv',
    f'itr_cia_aberta_BPP_ind_{ano}.csv',
    f'itr_cia_aberta_DFC_MD_ind_{ano}.csv',
    f'itr_cia_aberta_DFC_MI_ind_{ano}.csv',
    f'itr_cia_aberta_DMPL_ind_{ano}.csv',
    f'itr_cia_aberta_DRA_ind_{ano}.csv',
    f'itr_cia_aberta_DRE_ind_{ano}.csv',
    f'itr_cia_aberta_DVA_ind_{ano}.csv',
    # Composição de capital
    f'itr_cia_aberta_composicao_capital_{ano}.csv'
]

# Caminho do diretório do arquivo não zipado
destino_dir = f'C://B3//historico-arquivos//itr-b3//itr-cia-aberta//{ano}'

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    for arquivo in arquivos_a_extrair:
        zip_ref.extract(arquivo, destino_dir)

        csv_path = f'{destino_dir}//{arquivo}'
        parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

        # Tentativa de leitura com fallback
        try:
            df = pd.read_csv(
                csv_path, sep=';', encoding='ISO-8859-1',
                low_memory=False, engine='c'
            )
        except Exception as e:
            print(f"[AVISO] Problema ao ler {arquivo} com engine='c': {e}")
            print("→ Tentando novamente com engine='python' e on_bad_lines='warn'")
            df = pd.read_csv(
                csv_path, sep=';', encoding='ISO-8859-1',
                engine='python', on_bad_lines='warn'
            )

        # Salvando em parquet
        df.to_parquet(parquet_path)

# Entrando no diretório especificado
os.chdir(destino_dir)

# Encontrando todos os arquivos .csv no diretório
arquivos_csv = glob.glob('*.csv')

for arquivo in arquivos_csv:
    os.remove(arquivo)

In [24]:
# Compilando cada arquivo parquet em apenas um arquivo

# Configurações 
base_dir = Path(r'C:/B3/historico-arquivos/itr-b3/itr-cia-aberta')
years = list(range(2020, 2026))  # Lista dos anos de 2020 até 2025
output_dir = base_dir / 'consolidados'
output_dir.mkdir(parents=True, exist_ok=True)

# Encontrarndo todos os arquivos parquet nas pastas de cada ano
parquet_paths = []
for y in years:
    year_dir = base_dir / str(y)
    if not year_dir.exists():
        print(f'[AVISO] Diretório {year_dir} não encontrado — pulando.')
        continue
    parquet_paths.extend(list(year_dir.glob("*.parquet")))

if not parquet_paths:
    raise SystemExit('Nenhum arquivo parquet encontrado nas pastas de ano especificadas.')

# Agrupando por 'base name' (removendo o sufixo do ano se existir)
groups = {}
for p in parquet_paths:
    stem = p.stem  # ex: 'dfp_cia_aberta_BPA_con_2017'
    parts = stem.rsplit("_", 1)  # separa no último underscore
    if len(parts) == 2 and re.fullmatch(r"\d{4}", parts[1]):  # sufixo é um ano
        base_name = parts[0]
    else:
        # se não terminar em ano, usa o stem completo
        base_name = stem

    groups.setdefault(base_name, []).append(p)

# Para cada grupo, lendo todos os parquet e concatenando
for basename, paths in groups.items():
    # Ordenando por caminho (opcional: por ano)
    paths_sorted = sorted(paths, key=lambda x: x.name)
    print(f"\n[INFO] Agrupando {len(paths_sorted)} arquivos para '{basename}':")
    for pp in paths_sorted:
        print('  -', pp)

    dfs = []
    for pp in paths_sorted:
        try:
            df = pd.read_parquet(pp)
            dfs.append(df)
        except Exception as e:
            print(f'[ERRO] Falha ao ler {pp}: {e}. Pulando este arquivo.')
            continue

    if not dfs:
        print(f'[AVISO] Nenhum dataframe válido para {basename}, pulando.')
        continue

    # Concat — se colunas diferirem, concat preencherá com NaN
    try:
        df_all = pd.concat(dfs, ignore_index=True, sort=False)
    except Exception as e:
        print(f'[ERRO] Erro ao concatenar dataframes para {basename}: {e}')
        # tentativa alternativa: alinhar colunas manualmente
        common_cols = sorted({c for df in dfs for c in df.columns})
        dfs_aligned = [df.reindex(columns=common_cols) for df in dfs]
        df_all = pd.concat(dfs_aligned, ignore_index=True, sort=False)

    # Nome final do arquivo: 'basename_2017_2024.parquet'
    out_name = f'{basename}_{years[0]}_{years[-1]}.parquet'
    out_path = output_dir / out_name

    try:
        df_all.to_parquet(out_path, index=False)
        print(f'[OK] Gravado {out_path} (linhas: {len(df_all)}, colunas: {len(df_all.columns)})')
    except Exception as e:
        print(f'[ERRO] Falha ao salvar {out_path}: {e}')

print('\n[FINALIZADO] Todos os agrupamentos processados.')


[INFO] Agrupando 6 arquivos para 'itr_cia_aberta_BPA_con':
  - C:\B3\historico-arquivos\itr-b3\itr-cia-aberta\2020\itr_cia_aberta_BPA_con_2020.parquet
  - C:\B3\historico-arquivos\itr-b3\itr-cia-aberta\2021\itr_cia_aberta_BPA_con_2021.parquet
  - C:\B3\historico-arquivos\itr-b3\itr-cia-aberta\2022\itr_cia_aberta_BPA_con_2022.parquet
  - C:\B3\historico-arquivos\itr-b3\itr-cia-aberta\2023\itr_cia_aberta_BPA_con_2023.parquet
  - C:\B3\historico-arquivos\itr-b3\itr-cia-aberta\2024\itr_cia_aberta_BPA_con_2024.parquet
  - C:\B3\historico-arquivos\itr-b3\itr-cia-aberta\2025\itr_cia_aberta_BPA_con_2025.parquet
[OK] Gravado C:\B3\historico-arquivos\itr-b3\itr-cia-aberta\consolidados\itr_cia_aberta_BPA_con_2020_2025.parquet (linhas: 1053612, colunas: 14)

[INFO] Agrupando 6 arquivos para 'itr_cia_aberta_BPA_ind':
  - C:\B3\historico-arquivos\itr-b3\itr-cia-aberta\2020\itr_cia_aberta_BPA_ind_2020.parquet
  - C:\B3\historico-arquivos\itr-b3\itr-cia-aberta\2021\itr_cia_aberta_BPA_ind_2021.parquet